In [35]:
# system level
import os
import numpy as np
import pandas as pd
import time
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# machine learning
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import np_utils

from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, brier_score_loss

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import matplotlib.patches as mpatches

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'
import matplotlib
%matplotlib inline

### Load data

In [9]:
# Load year 1 train, test, validation image files
X_train_1 = np.load('images_Y1_train.npy')
X_test_1 = np.load('images_Y1_test.npy')
X_valid_1 = np.load('images_Y1_valid.npy')

In [10]:
# Load year 10 train, test, validation image files
X_train_10 = np.load('images_Y10_train.npy')
X_test_10 = np.load('images_Y10_test.npy')
X_valid_10 = np.load('images_Y10_valid.npy')

In [12]:
# Load year 1 and 10, label small subset test files
X_test_1_small = np.load('images_Y1_test_150.npy')
X_test_10_small = np.load('images_Y10_test_150.npy')
Y_test_small = np.load('labels_test_150.npy')

In [13]:
# Load image label files
Y_train = np.load('labels_train.npy')
Y_test = np.load('labels_test.npy')
Y_valid = np.load('labels_valid.npy')

Y_train_oh = tf.keras.utils.to_categorical(Y_train)
Y_test_oh = tf.keras.utils.to_categorical(Y_test)

In [25]:
Y_train_df = pd.DataFrame(Y_train)
Y_test_df = pd.DataFrame(Y_test)
Y_valid_df = pd.DataFrame(Y_valid)

In [15]:
# training variables
# 70:10:20 for training:valid:test
NUM_TRAIN = 23487 # num train images
NUM_TEST = 6715 # num test images
NUM_VALIDATION = 3355 # num validation images
shuffle = True

In [42]:
# check data sizes
NUM_TOTAL = NUM_TRAIN + NUM_TEST + NUM_VALIDATION
print(NUM_TOTAL)
assert NUM_TOTAL == len(X_train_1) + len(X_test_1) + len(X_valid_1), "total training, test, validation samples not equal to total samples - exiting"


33557


### EDA

In [60]:
#Y_train_df.info()
#Y_test_df.info()
#Y_valid_df.info()

In [56]:
#Y_train_df.value_counts()
#Y_test_df.value_counts()
#Y_valid_df.value_counts()

In [53]:
#Y_train_df.describe()
#Y_test_df.describe()
#Y_valid_df.describe()

In [38]:
print( "Data dimensions: ") 
print( "Training Set (year 1): ", np.shape(X_train_1), np.shape(Y_train)) # same for year 10
print( "Test Set (year 1): ", np.shape(X_test_1), np.shape(Y_test)) # same for year 10
print( "Validation Set (year 1): ", np.shape(X_valid_1), np.shape(Y_valid)) # same for year 10

Data dimensions: 
Training Set (year 1):  (23487, 3, 100, 100) (23487, 3)
Test Set (year 1):  (6715, 3, 100, 100) (6715, 3)
Validation Set (year 1):  (3355, 3, 100, 100) (3355, 3)


In [128]:
data = Y_train_df.set_axis(['Spiral', 'Elliptical', 'Merger'], axis=1, inplace=False)

In [39]:
# g = sns.FacetGrid(data, col="Spiral", hue="Spiral")
# g.map(sns.countplot, "Spiral");
# g.add_legend(labels=["No", "Yes"]);

In [40]:
# g = sns.FacetGrid(data, col="Elliptical", hue="Elliptical")
# g.map(sns.countplot, "Elliptical");
# g.add_legend(labels=["No", "Yes"]);

In [41]:
# g = sns.FacetGrid(data, col="Merger", hue="Merger")
# g.map(sns.countplot, "Merger");
# g.add_legend(labels=["No", "Yes"]);

```
3 columns: spiral (or '0'), elliptical ('1'), or a galaxy merger ('2’)
No null values, min:0, max:1

Training Set: 
23487 images,
dtype: int64,
shape: (23487, 3, 100, 100)

Training labels:

spiral: 
	10017 total,
	mean: 0.426491,
	std: 0.494577

elliptical: 
	5705 total,
	mean: 0.242900,
	std: 0.428844

merger:
	7765 total
	mean: 0.330608
	std: 0.470442

Test Set: 
6715 images,
dtype: int64,
shape: (6715, 3, 100, 100)

Test labels:
spiral: 
	2863 total,
	mean: 0.426359,
	std: 0.494584

elliptical:
	1631 total,
	mean: 0.242889,
	std: 0.428861

merger:
	2221 total,
	mean: 0.330752,
	std: 0.470519

Validation Set: 
3355 images,
dtype: int64,
shape: (3355, 3, 100, 100)

Validation labels:
spiral: 
	1432 total,
	mean: 0.426826,
	std: 0.494690

elliptical:
	815 total,
	mean: 0.242921,
	std: 0.428912

merger:
	1108 total,
	mean: 0.330253,
	std: 0.470374
```

### Build and train a CNN model:
First train a standard deterministic CNN classifier model as a baseline model

In [167]:
print("Tensorflow Version: ", tf.__version__)
print("Tensorflow Probability Version: ", tfp.__version__)

Tensorflow Version:  2.8.0
Tensorflow Probability Version:  0.16.0


In [227]:
learning_rate = 0.001 # default initial learning rate
nb_epoch = 500

In [ ]:
data_shape = np.shape(x_data)
input_shape = (2, 75, 75)

x = Input(shape=input_shape)
c0 = Convolution2D(8, (5, 5), activation='relu', strides=(1, 1), padding='same', data_format='channels_first')(x)
b0 = BatchNormalization()(c0)
d0 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first')(b0)
e0 = Dropout(0.5)(d0)

c1 = Convolution2D(16, (3, 3), activation='relu', strides=(1, 1), padding='same', data_format='channels_first')(e0)
b1 = BatchNormalization()(c1)
d1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first')(b1)
e1 = Dropout(0.5)(d1)

c2 = Convolution2D(32, (3, 3), activation='relu', strides=(1, 1), padding='same', data_format='channels_first')(e1)
b2 = BatchNormalization()(c2)
d2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first')(b2)
e2 = Dropout(0.5)(d2)

f = Flatten()(e2)
z0 = Dense(64, activation='softmax', kernel_regularizer=l2(0.0001))(f)
z1 = Dense(32, activation='softmax', kernel_regularizer=l2(0.0001))(z0)
y = Dense(1, activation='sigmoid')(z1)

model = Model(inputs=x, outputs=y)

# Compile Model
optimizer = 'adam'
metrics = ['accuracy']
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
model.summary()

### BNN

In [180]:
# model = tf.keras.models.Sequential([
    
#     tfp.layers.Convolution2DFlipout(
#         6, kernel_size=5, padding='SAME', 
#         kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.relu),
    
#     tf.keras.layers.MaxPooling2D(
#         pool_size=[2, 2], strides=[2, 2],
#         padding='SAME'),

#     tfp.layers.Convolution2DFlipout(
#         16, kernel_size=5, padding='SAME', 
#         kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.relu),
    
#     tf.keras.layers.MaxPooling2D(
#         pool_size=[2, 2], strides=[2, 2],
#         padding='SAME'),
    
#     tfp.layers.Convolution2DFlipout(
#         120, kernel_size=5, padding='SAME', 
#         kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.relu),
    
#     tf.keras.layers.Flatten(),
    
#     tfp.layers.DenseFlipout(
#         84, kernel_divergence_fn=kl_divergence_function,
#         activation=tf.nn.relu),
    
#     tfp.layers.DenseFlipout(
#         NUM_CLASSES, kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.softmax)
# ]
# )

In [157]:
# # model inputs
# FEATURE_NAMES = [
#     "Spiral",
#     "Elliptical",
#     "Merger"
# ]


# def create_model_inputs():
#     inputs = {}
#     for feature_name in FEATURE_NAMES:
#         inputs[feature_name] = layers.Input(
#             name=feature_name, shape=(1,), dtype=tf.int64
#         )
#     return inputs

In [159]:
# def run_experiment(model, loss, train_dataset, test_dataset):

#     model.compile(
#         optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
#         loss=loss,
#         metrics=[keras.metrics.RootMeanSquaredError()],
#     )

#     print("Start training the model...")
#     model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
#     print("Model training finished.")
#     _, rmse = model.evaluate(train_dataset, verbose=0)
#     print(f"Train RMSE: {round(rmse, 3)}")

#     print("Evaluating model performance...")
#     _, rmse = model.evaluate(test_dataset, verbose=0)
#     print(f"Test RMSE: {round(rmse, 3)}")